In [90]:
from transformers import AutoModelForSequenceClassification
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, DataCollatorWithPadding, Trainer, EarlyStoppingCallback
import evaluate
import warnings
import torch

torch.manual_seed(42)
warnings.filterwarnings("ignore")

In [91]:
model_name = "distilbert-base-uncased"

In [92]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [93]:
data_files = {"train": "binaryRating_title_train.csv", "test": "binaryRating_title_test.csv"}

dataset = load_dataset("csv", data_files=data_files)

dataset = dataset.filter(lambda x: x["title"] is not None and str(x["title"]).strip() != "")

def tokenize_function(set):
    # Handle None values and ensure all titles are strings
    return tokenizer(set["title"], padding="max_length", truncation=True, max_length=122) # 122 is the max length for all sentences in train and test dataset
    

dataset = dataset.map(tokenize_function, batched=True)

dataset["train"] = dataset["train"].select(range(100000))

labels = [ "1", "2" ]

def label_mapping(x):
    return labels.index(str(x))

dataset = dataset.map(lambda x: {"label": label_mapping(x["rating"])})

Map:   0%|          | 0/399976 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [94]:
# Initialize a BERT model for binary classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=9)
model.config.id2label = {i: label for i, label in enumerate(labels)}
model.config.label2id = {label: i for i, label in enumerate(labels)}

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [96]:
# Freeze all layers except the classifier
for param in model.distilbert.parameters():
    param.requires_grad = True

In [97]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,             
    per_device_train_batch_size=128, 
    per_device_eval_batch_size=128,
    use_mps_device=True,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=10,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch"
)

In [98]:
metric = evaluate.load("accuracy")

In [99]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [100]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,                        # Pre-trained BERT model
    args=training_args,                 # Training arguments
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,        # Efficient batching
    compute_metrics=compute_metrics     # Custom metric function
)
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))


In [101]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

{'eval_loss': 0.26071134209632874,
 'eval_accuracy': 0.887,
 'eval_runtime': 59.4719,
 'eval_samples_per_second': 168.147,
 'eval_steps_per_second': 1.328,
 'epoch': 2.0}

In [ ]:
D = dataset["test"].select([10]).remove_columns(["Segment", "Article"])
predictions = trainer.predict(D)
predicted_class = predictions.predictions.argmax(axis=-1)[0]
print(f"Predicted class: {labels[predicted_class]} (index: {predicted_class})")

ValueError: Column name ['Article', 'Segment'] not in the dataset. Current columns in the dataset: ['rating', 'title', 'input_ids', 'token_type_ids', 'attention_mask', 'label']

In [ ]:
from transformers import pipeline

# Create a text classification pipeline using the trained model
classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=False,
    device="mps" if torch.backends.mps.is_available() else "cpu"
)

# Function to classify text(s) and return label names
def classify_text(texts):
    if isinstance(texts, str):
        texts = [texts]
    
    results = classifier(texts)
    return [result['label'] for result in results]

Device set to use mps


In [ ]:
classifier(["Der angekündigte Umbau des Landtagsklubs schreitet zäh voran, in der Stadt ist kein logischer Nachfolger für Bürgermeister Schaden in Sicht. Er hat ein bisserl warten müssen, aber im Dezember dürfte er den Sprung in den Landtag schaffen: Tarik Mete, 28 Jahre alt und eine der wenigen echten Nachwuchshoffnungen der Salzburger SPÖ. Der rote Jungstar mit türkischen Wurzeln, der bei der Landtagswahl 2013 immerhin 1.800 Vorzugsstimmen erreichte, soll das Mandat von Nicole Solarz (34) übernehmen, die in Karenz gehen wird. Derzeit ist Mete als Assistent des Obmanns bei der Salzburger Gebietskrankenkasse beschäftigt. Landesparteiobmann und Landtagsklubobmann Walter Steidl bestätigt im STANDARD-Gespräch entsprechende Pläne. Damit sei der zweite personalpolitische Parteivorstandsbeschluss umgesetzt, sagt Steidl. Dieser habe Metes Karriere betroffen, der andere den Bezirk Lungau. Nach dem aus privaten Gründen erfolgten Rückzug des Schwarzacher Bürgermeisters Andreas Haitzer aus dem Landtag ist der 1972 geborene Bürgermeister von St. Margarethen, Gerd Brand, nachgerückt. Der von Steidl bald nach der Wahl 2013 angekündigte Umbau des Landtagsklubs schreitet mit dem Nachrücken Metes nun zwar voran, allerdings zäh. Die lange Zeit als Ablösekandidatin gehandelte zweite Landtagspräsidentin Gudrun Mosler-Törnström (59) wird wohl bis zur nächsten Wahl bleiben. Offen ist, ob die Landesgeschäftsführerin des ÖGB, Heidi Hirschbichler (56), sich früher aus dem Landtag zurückzieht. Mittelfristig ist für die Sozialdemokraten an der Salzach freilich die Frage wesentlich bedeutsamer, wer Langzeitbürgermeister Heinz Schaden nachfolgen soll. Schaden ist seit 1992 Mitglied der Stadtregierung und seit 1999 Bürgermeister. Der 61-Jährige hat wiederholt angekündigt, bei der Bürgermeister- und Gemeinderatswahl im März 2019 nicht mehr anzutreten. Eine Entscheidung über seine Nachfolge als SPÖ-"])

[{'label': 'Inland', 'score': 0.627457857131958}]

In [ ]:
import pickle

# Save the classifier pipeline to a file
with open('bert_classifier_pipeline.jl', 'wb') as f:
    pickle.dump(classifier, f)

print("Pipeline saved to 'bert_classifier_pipeline.jl'")

classifier.save_pretrained("bert_classifier_pipeline")
print("Pipeline saved to 'bert_classifier pipeline'")

Pipeline saved to 'bert_classifier_pipeline.jl'
Pipeline saved to 'bert_classifier pipeline'
